In [8]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [9]:
# making a function to get the middle portion of the location out because that is what was causing errors
def split_string(location):
    string = list(location.split(','))
    del string[1]
    new_string = ''
    for e in string:
        new_string += e +','
    new_string = new_string[:-1]
    return new_string
print(split_string('Paris, Ile-de-France, France'))
print(split_string('Sydney, New South Wales, Australia'))

Paris, France
Sydney, Australia


In [10]:
data = pd.read_csv('/Users/rohanmahajan/Downloads/ufc_event_details (1).csv')
data.LOCATION.head(10)

0                        Las Vegas, Nevada, USA
1                  New York City, New York, USA
2                  Sao Paulo, Sao Paulo, Brazil
3    Abu Dhabi, Abu Dhabi, United Arab Emirates
4                        Las Vegas, Nevada, USA
5                        Las Vegas, Nevada, USA
6                        Las Vegas, Nevada, USA
7                        Las Vegas, Nevada, USA
8            Sydney, New South Wales, Australia
9                  Paris, Ile-de-France, France
Name: LOCATION, dtype: object

In [11]:
data.LOCATION = data.LOCATION.apply(split_string)
data

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Allen vs. Craig,http://ufcstats.com/event-details/3feed7d02711...,"November 18, 2023","Las Vegas, USA"
1,UFC 295: Prochazka vs. Pereira,http://ufcstats.com/event-details/5a558ba1ff5e...,"November 11, 2023","New York City, USA"
2,UFC Fight Night: Almeida vs. Lewis,http://ufcstats.com/event-details/7c4ec656d8fc...,"November 04, 2023","Sao Paulo, Brazil"
3,UFC 294: Makhachev vs. Volkanovski 2,http://ufcstats.com/event-details/13a0fb8fbdaf...,"October 21, 2023","Abu Dhabi, United Arab Emirates"
4,UFC Fight Night: Yusuff vs. Barboza,http://ufcstats.com/event-details/f3a078277b3b...,"October 14, 2023","Las Vegas, USA"
...,...,...,...,...
665,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, USA"
666,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, USA"
667,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, USA"
668,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, USA"


In [12]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyGeocodingApp/1.0")
data['lat'] = -float('inf')
data['long'] = -float('inf')
locations_list = data.LOCATION
for i in range(len(locations_list)):
    location_info = geolocator.geocode(locations_list[i])
    #time.sleep(1)  # Introduce a delay to comply with rate limits
    data.at[i, 'lat'] = location_info.latitude
    data.at[i, 'long'] = location_info.longitude
        

In [13]:
data.head(10)

,EVENT,URL,DATE,LOCATION,lat,long
0,UFC Fight Night: Allen vs. Craig,http://ufcstats.com/event-details/3feed7d02711...,"November 18, 2023","Las Vegas, USA",36.167256,-115.148516
1,UFC 295: Prochazka vs. Pereira,http://ufcstats.com/event-details/5a558ba1ff5e...,"November 11, 2023","New York City, USA",40.712728,-74.006015
2,UFC Fight Night: Almeida vs. Lewis,http://ufcstats.com/event-details/7c4ec656d8fc...,"November 04, 2023","Sao Paulo, Brazil",-23.550651,-46.633382
3,UFC 294: Makhachev vs. Volkanovski 2,http://ufcstats.com/event-details/13a0fb8fbdaf...,"October 21, 2023","Abu Dhabi, United Arab Emirates",24.453835,54.377401
4,UFC Fight Night: Yusuff vs. Barboza,http://ufcstats.com/event-details/f3a078277b3b...,"October 14, 2023","Las Vegas, USA",36.167256,-115.148516
5,UFC Fight Night: Dawson vs. Green,http://ufcstats.com/event-details/c8a49ff2acb6...,"October 07, 2023","Las Vegas, USA",36.167256,-115.148516
6,UFC Fight Night: Fiziev vs. Gamrot,http://ufcstats.com/event-details/c945adc22c2b...,"September 23, 2023","Las Vegas, USA",36.167256,-115.148516
7,UFC Fight Night: Grasso vs. Shevchenko 2,http://ufcstats.com/event-details/8fa2b0657236...,"September 16, 2023","Las Vegas, USA",36.167256,-115.148516
8,UFC 293: Adesanya vs. Strickland,http://ufcstats.com/event-details/ece280745f87...,"September 09, 2023","Sydney, Australia",-33.768528,150.956856
9,UFC Fight Night: Gane vs. Spivac,http://ufcstats.com/event-details/ef61d9f5176b...,"September 02, 2023","Paris, France",48.853495,2.348391


In [14]:
import folium
from folium.plugins import HeatMap
# create base map object centered at the center of the world
mapObj = folium.Map(location=[39, 34], zoom_start = 2)
# create heatmap layer
heatmap = HeatMap( list(zip(data['lat'], data['long'])),
                   min_opacity=0.25,
                   radius=55, blur=35, 
                   max_zoom=1)
# add heatmap layer to base map
heatmap.add_to(mapObj)
mapObj
